<a href="https://colab.research.google.com/github/crisdavid3335/analytics_with_KERAS/blob/main/StoreSalesTimeSeriesForecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importamos las librerías a utilizar
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb
from xgboost import XGBRegressor

import tensorflow as tf
import keras 
from keras import layers
from keras.losses import mean_squared_error

from datetime import date, datetime
import time
import calendar

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Cargamos los datos
train = pd.read_csv('/content/drive/MyDrive/Proyectos/Store Sales - Time Series Forecasting/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Proyectos/Store Sales - Time Series Forecasting/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/Proyectos/Store Sales - Time Series Forecasting/sample_submission.csv')

oil = pd.read_csv('/content/drive/MyDrive/Proyectos/Store Sales - Time Series Forecasting/oil.csv')
holiday = pd.read_csv('/content/drive/MyDrive/Proyectos/Store Sales - Time Series Forecasting/holidays_events.csv')
store = pd.read_csv('/content/drive/MyDrive/Proyectos/Store Sales - Time Series Forecasting/stores.csv')
tran = pd.read_csv('/content/drive/MyDrive/Proyectos/Store Sales - Time Series Forecasting/transactions.csv')

In [3]:
# Procesamos los datos 

# Vamos a añadir los días de la semana, los meses, los años, el día fecha y los días de paga
def pre_train(df):
  '''
  Esta función agrega las convenciones de tiempo.
  '''
  df['date'] = df['date'].map(lambda x: date.fromisoformat(x))
  df['weekday'] = df['date'].map(lambda x: x.weekday())
  df['year'] = df['date'].map(lambda x: x.year)
  df['month'] = df['date'].map(lambda x: x.month)
  df['day'] = df['date'].map(lambda x: x.day)
  df['eomd'] = df['date'].map(lambda x: calendar.monthrange(x.year, x.month)[1])
  df['payday'] = ((df['day'] == df['eomd'])|(df['day'] == 15)).astype(int)
  df.drop(['id', 'eomd'], axis = 1, inplace=True)
  return df

train = pre_train(train)
test = pre_train(test)

In [4]:
# Vamos a reemplazar los valores nulos del precio del petróleo por su promedio mensual

def pre_oil(df):
  df['month'] = df['date'].map(lambda x: int(x.replace('-', '')[:6]))
  df['month_avg'] = df.groupby('month')['dcoilwtico'].transform('mean')
  df['tmp'] = df['dcoilwtico'].map(np.isnan)
  df['month_avg'] = df['tmp'] * df['month_avg']
  df['dcoilwtico'].fillna(0, inplace =True)
  df['dcoilwtico'] = df['dcoilwtico'] + df['month_avg']
  df = df.drop(['month', 'month_avg', 'tmp'], axis = 1)
  df['date'] = df['date'].map(lambda x: date.fromisoformat(x))
  return df

oil = pre_oil(oil)

In [5]:
# Procesamos los días feriado, vacaiones, etc.
def pre_holyday(df):
  df['date'] = df['date'].map(lambda x: date.fromisoformat(x))
  df = df[(df['transferred'] == False) & (df['type'] != 'Work day')]
  event = df[df['type'] == 'Event']
  earthquake = event[event['description'].str.startswith('Terremoto Manabi')]
  event = event[event['description'].str.startswith('Terremoto Manabi') == False]
  return df, event, earthquake

holiday, event, earthquake = pre_holyday(holiday)

In [6]:
# Separamos los tipos de eventos
event = event[['date', 'description']]
event.rename({'description': 'event_name'}, axis = 1, inplace=True)

earthquake = earthquake[['date', 'description']]
earthquake.rename({'description': 'earthquake'}, axis = 1, inplace=True)

h_local = holiday[holiday['locale'] == 'Local']
h_local = h_local[['date', 'locale_name', 'description']]
h_local = h_local.rename({'locale_name': 'city', 'description': 'local_holiday_name'}, axis = 1)

h_regional = holiday[holiday['locale'] == 'Regional']
h_regional = h_regional[['date', 'locale_name', 'description']]
h_regional = h_regional.rename({'locale_name' : 'state', 
                                'description': 'regional_holiday_name'}, axis = 1)

h_national = holiday[holiday['locale'] == 'National']
h_national = holiday[['date', 'description']]
h_national = h_national.rename({'description': 'national_holiday_name'}, axis = 1)

In [7]:
# Vamos a unir los conjuntos de datos 
def merge_tables(df):
  df = df.merge(oil, on = 'date', how = 'left')
  df = df.merge(store, on = 'store_nbr', how='left')
  df = df.merge(event, on = 'date', how='left').fillna('0')
  df = df.merge(earthquake, on = 'date', how='left').fillna('0')
  df = df.merge(h_local, on = ['date', 'city'], how='left').fillna('0')
  df = df.merge(h_regional, on = ['date', 'state'], how='left').fillna('0')
  df = df.merge(h_national, on = 'date', how='left').fillna('0')
  df = df.merge(tran, on = ['date', 'store_nbr'], how='left').fillna('0')
  return df

train = merge_tables(train)
test = merge_tables(test)

In [8]:
# Cambiamos el tipo de datos a decimales
train['dcoilwtico'] = train['dcoilwtico'].astype(float)
test['dcoilwtico'] = test['dcoilwtico'].astype(float)

In [9]:
# Codificamos las caracteristicas categoricas
cat_features = ['family', 'store_nbr', 'city', 'state', 'type', 'cluster', 
                'event_name', 'earthquake', 'local_holiday_name', 
                'regional_holiday_name', 'national_holiday_name']
for col in cat_features:
  label_enc = LabelEncoder()
  train[col] = label_enc.fit_transform(train[col])
  test[col] = label_enc.transform(test[col])

In [10]:
# Vamos a separar los datos de entrenamiento y validación
# Como es una serie de tiempo se debe hacer secuencial
# Seran 31 días de validación
def pre_data(df, train_date: list, valid_date: list):
  df['is_train'] = df['date'].map(lambda x: x in train_date)
  df['is_val'] = df['date'].map(lambda x: x in valid_date)
  return df

train_date = train['date'].unique()[-227: -31].tolist()
valid_date = train['date'].unique()[-31:].tolist()
train = pre_data(train, train_date, valid_date)

In [11]:
print(f'Los datos de entrenamiento empiezan el {min(train_date)} y terminan el {max(train_date)}.')
print(f'El periodo de prueba empieza el {min(valid_date)} y termina el {max(valid_date)}.')

Los datos de entrenamiento empiezan el 2017-01-01 y terminan el 2017-07-15.
El periodo de prueba empieza el 2017-07-16 y termina el 2017-08-15.


In [12]:
# Generamos los conjuntos de datos y objetivos (X, y).
# Para normalizar las ventas se toma su logaritmo
y = np.log(train['sales'] + 1)

X_train = train.drop(['date', 'sales', 'year'], axis = 1)
X_test = test.drop(['date', 'year'], axis = 1).values.astype(float)

trn_idx = X_train[X_train['is_train']==True].index.tolist()
val_idx = X_train[X_train['is_val']==True].index.tolist()

X_tr = X_train.loc[trn_idx, :].drop(['is_train', 'is_val'], axis=1).values.astype(float)
X_val = X_train.loc[val_idx, :].drop(['is_train', 'is_val'], axis=1).values.astype(float)
y_tr = y[trn_idx].values.astype(float)
y_val = y[val_idx].values.astype(float)

In [13]:
# Construimos los modelos

# DNN
dnn_model = keras.Sequential([
                              layers.Dense(256, activation = 'relu'),
                              layers.Dropout(0.3),
                              layers.Dense(256, activation = 'relu'),
                              layers.Dropout(0.3),
                              layers.Dense(128, activation = 'relu'),
                              layers.Dense(128, activation = 'relu'),
                              layers.Dense(1)
])

dnn_model.compile(optimizer = 'adam', 
              loss = mean_squared_error,
              metrics = ['mae'])

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    mode = 'auto',    
    patience = 50,
    verbose = 0, 
    restore_best_weights = True)

# Xgboost
xgb_model = XGBRegressor(max_depth = 5, verbosity = 0, 
                         objective = 'reg:squarederror', 
                         n_estimators = 600, 
                         reg_alpha=10., subsample = 0.9963,
                         min_child_weight=47, 
                         random_state = 0)

In [14]:
# Entrenamos los modelos
history = dnn_model.fit(X_tr,
                    y_tr,
                    epochs = 400,
                    batch_size = 128,
                    callbacks = [early_stopping],
                    validation_data = (X_val, y_val))

xgb_model.fit(X_tr, y_tr,
              early_stopping_rounds = 100, 
              eval_set=[(X_val, y_val)])

Epoch 1/400
2785/2785 [==============================] - 25s 9ms/step - loss: 2.6027 - mae: 1.2749 - val_loss: 2.5911 - val_mae: 1.2798
Epoch 2/400
2785/2785 [==============================] - 27s 10ms/step - loss: 1.8871 - mae: 1.0637 - val_loss: 2.1271 - val_mae: 1.1043
Epoch 3/400
2785/2785 [==============================] - 27s 10ms/step - loss: 1.7113 - mae: 1.0000 - val_loss: 1.8531 - val_mae: 1.0129
Epoch 4/400
2785/2785 [==============================] - 28s 10ms/step - loss: 1.6193 - mae: 0.9622 - val_loss: 1.7895 - val_mae: 1.0019
Epoch 5/400
2785/2785 [==============================] - 22s 8ms/step - loss: 1.5611 - mae: 0.9366 - val_loss: 1.9574 - val_mae: 1.0301
Epoch 6/400
2785/2785 [==============================] - 21s 8ms/step - loss: 1.5267 - mae: 0.9209 - val_loss: 1.7732 - val_mae: 0.9533
Epoch 7/400
2785/2785 [==============================] - 21s 8ms/step - loss: 1.5037 - mae: 0.9108 - val_loss: 1.6282 - val_mae: 0.9364
Epoch 8/400
2785/2785 [======================

XGBRegressor(max_depth=5, min_child_weight=47, n_estimators=600,
             objective='reg:squarederror', reg_alpha=10.0, subsample=0.9963,
             verbosity=0)

In [15]:
# Usamos estos modelos para predecir las caracteristicas para un nuevo modelo
def ensemble_model(data_set):
  dnn_y_hat = dnn_model.predict(data_set)
  xgb_y_hat = xgb_model.predict(data_set)

  dnn_y_hat = dnn_y_hat.reshape(len(dnn_y_hat),)

  X_global =  pd.DataFrame({'Dnn_predict':dnn_y_hat, 
                            'Xgboost_predict': xgb_y_hat})
  X_global['Dnn_predict'] = X_global['Dnn_predict'].where(X_global['Dnn_predict'] >0, 0)
  X_global['Xgboost_predict'] = X_global['Xgboost_predict'].where(X_global['Xgboost_predict'] >0, 0)
  return X_global

In [16]:
X_global = ensemble_model(X_tr)
X_global

,Dnn_predict,Xgboost_predict
0,0.257431,0.000000
1,0.169561,0.000000
2,0.215348,0.000000
3,0.271430,0.406273
4,0.275013,0.000000
...,...,...
356395,3.232460,6.214194
356396,1.868192,4.823900
356397,7.384979,7.714790
356398,1.582748,1.186668


In [17]:
# Se usan las predicciones anteriores para generar el nuevo modelo
global_model = RandomForestRegressor(random_state = 0)
global_model.fit(X_global, y_tr)

X_test_ensemble = ensemble_model(X_test)
y_hat = global_model.predict(X_test_ensemble)

In [18]:
# Generamos la predicción global 
sub['sales'] = np.exp(y_hat) - 1
sub['sales'] = sub['sales'].where(sub['sales']>0, 0)
sub.to_csv('submission.csv', index=False)

In [25]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

X_global = ensemble_model(X_val)
y_hat = global_model.predict(X_global)

mae = mean_absolute_error(y_val, y_hat)
mse = mean_squared_error(y_val, y_hat)
r2_scor = r2_score(y_val, y_hat)

In [28]:
print('MAE score: %.3f' % mae)
print('MSE score: %.3f' % mse)
print('R2 score: %.3f' % r2_scor)

MAE score: 0.356
MSE score: 0.252
R2 score: 0.961
